# Import packages

In [2]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
# -*- coding:utf-8 -*-

# 将标注的json文件合并为Excel

In [3]:
# 主要功能：合并json文件中的标注为Excel
# 次要功能：将标注选取文本过多的标注裁剪，尤其是Evidence
# 次要功能：当有多个Evidence时，选取证据等级最高的一个
input_path = "input"
col_dic =  {"证据等级":"evi", "样本量":"num","患者性别":"gender","年龄":"age","免疫状态":"immu"}
col_dic = {"是否致病":"pat"}
sped_l = [ele for ele in os.listdir(input_path) if "." not in ele]


for col_name, col_ab in col_dic.items():
    
    for sped in sped_l:
        
        collection = {}
        paper_l = [ i for i in os.listdir("%s/%s/outputs" % (input_path, sped)) if "json" in i ]
        
        for paper in paper_l:
            
            # pattern 用于储存信息
            pattern = {
                "物种名称": "",
                "是否是致病菌": "",
                "样本量": "",
                "患者性别": "",
                "年龄": "",
                "免疫状态": "",
                "预后": "",
                "肺部基础疾病": "",
                "证据等级": "",
            }
            
            # 读取json文件
            with open("%s/%s/outputs/%s" % (input_path, sped, paper),
                    "r",encoding='UTF-8') as f:
                ano = json.load(f)
                
            evi_list = []
            evis_list = []
            
            pattern["物种名称"] = ano["content"].split("\r\n")[0][8:]
            if ano["outputs"] != {}:
                
                # ["A"]为标注的类别
                for i in ano["outputs"]["annotation"]["A"]:
                    
                    if len(i) > 1:
                        
                        # 保存所有信息到pattern
                        if i["name"] in pattern.keys():
                            pattern[i["name"]] = i["value"]
                            
                        # 获取用于判断证据等级大小的信息
                        if i["name"]  in col_name:
                            evi_list.append(i["value"])
                            
                # ["T"]为标注对应的类别的文本
                for j in ano["outputs"]["annotation"]["T"]:
                    
                    if (j is not None) and (len(j) > 1):
                            
                        # 获取标注的判断证据等级的文本
                        if j["name"] in col_name:
                            evis_list.append(j["value"])
                            
                            # 比较等级
                            if len(evis_list) > 1:
                                
                                evi_new = evi_list[len(evis_list) - 1]
                                evi_old = evi_list[len(evis_list) - 2]
                                
                                if evi_new < evi_old:
                                    
                                    pattern[col_name] == j["value"]
                                    pattern["answer"] = j["value"]
                                    pattern["old_start"] = j["start"]
                                    pattern["old_end"] = j["end"]
                                    
                                    # 获取证据等级所在的整个句子
                                    anr = ano["content"].replace("\r\n", "  ")
                                    new_start = [
                                        anr.rfind(
                                            "%s" % k, 0, j["start"])
                                        for k in string.punctuation
                                        if anr.rfind(
                                            "%s" % k, 0, j["start"]) > 0
                                    ]
                                    new_start.append(0)
                                    new_start = np.max(new_start)
                                    new_end = [
                                        anr.find(
                                            "%s" % k, j["end"], -1)
                                        for k in string.punctuation
                                        if anr.find(
                                            "%s" % k, j["end"], -1) > 0
                                    ]
                                    new_end.append(len(anr) - 1)
                                    new_end = np.min(new_end)
                                    
                                    evi_txt = anr[new_start +
                                                            1:new_end + 1]
                                    pattern["sup_evi"] = evi_txt
                                    pattern["new_start"] = new_start + 1
                                    pattern["new_end"] = new_end + 1
                                    
                            else:
                                pattern["answer"] = j["value"]
                                pattern["old_start"] = j["start"]
                                pattern["old_end"] = j["end"]
                                
                                anr = ano["content"].replace("\r\n", "  ")
                                new_start = [
                                        anr.rfind(
                                            "%s" % k, 0, j["start"])
                                        for k in string.punctuation
                                        if anr.rfind(
                                            "%s" % k, 0, j["start"]) > 0
                                    ]
                                new_start.append(0)
                                new_start = np.max(new_start)
                                new_end = [
                                    anr.find(
                                        "%s" % k, j["end"], -1)
                                    for k in string.punctuation
                                    if anr.find(
                                        "%s" % k, j["end"], -1) > 0
                                ]
                                new_end.append(len(anr) - 1)
                                new_end = np.min(new_end)
                                
                                evi_txt = anr[new_start +
                                                        1:new_end + 1]
                                pattern["sup_evi"] = evi_txt
                                pattern["new_start"] = new_start + 1
                                pattern["new_end"] = new_end + 1
                                
            pattern["full"] = ano["content"]
            pattern["id"] = 'https://pubmed.ncbi.nlm.nih.gov/%s/' % paper.replace(".json", "")
            
            collection[paper.replace(".json", "")] = pattern
            
        df = pd.DataFrame(collection).T
        print(df.shape)
        df.to_excel("output/文献标注/%s_%s.xls" % (sped, col_ab))

(436, 11)
(78, 11)


/tmp/ipykernel_713518/560058268.py:143: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("output/文献标注/%s_%s.xls" % (sped, col_ab))


(690, 11)
(690, 11)
(289, 11)
(153, 11)
(153, 11)
(221, 11)
(221, 11)
(78, 11)
(436, 11)
(289, 11)
